# Data Cleaning Notebook
- utilize insights from EDA notebook to clean data
- construct target field
- drop redundant and highly correlated fields
- handle null/missing values
- log transform fields with heavy tails

In [37]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv("../data/bitcoin.csv", parse_dates=["Date"])
df = df.sort_values("Date").reset_index(drop=True)

In [41]:
# trim early zero price period
df = df[df['btc_market_price'] > 0].copy()

# construct t+7 target
df['target_t7'] = df['btc_market_price'].shift(-7)
df = df.dropna(subset=['target_t7'])

In [43]:
# drop redundant / irrelevant fields
drop_cols = [
    "btc_market_cap",
    "btc_estimated_transaction_volume_usd",
    "btc_cost_per_transaction_percent",
    "btc_n_transactions_total",
    "btc_n_orphaned_blocks",  
]
df = df.drop(columns=drop_cols, errors="ignore")

# collapse correlated groups
df = df.drop(columns=[
    "btc_blocks_size",  # keep avg_block_size
    "btc_difficulty",   # keep hash_rate
    "btc_miners_revenue", # keep transaction_fees
    "btc_n_transactions_excluding_popular",
    "btc_n_transactions_excluding_chains_longer_than_100",
    "btc_n_transactions_per_block"
], errors="ignore")

In [45]:
# impute missing btc_trade_volume values
["btc_trade_volume"] = df["btc_trade_volume"].ffill()

In [49]:
# log transform heavy tailed fields
log_cols = [
    "btc_trade_volume",
    "btc_transaction_fees",
    "btc_output_volume",
    "btc_estimated_transaction_volume"
]
for col in log_cols:
    if col in df.columns:
        df[col + "_log"] = np.log1p(df[col])

In [51]:
# save cleaned dataset
df.to_csv("../data/bitcoin_clean.csv", index=False)

df.head(10)

,Date,btc_market_price,btc_total_bitcoins,btc_trade_volume,btc_avg_block_size,btc_median_confirmation_time,btc_hash_rate,btc_transaction_fees,btc_cost_per_transaction,btc_n_unique_addresses,btc_n_transactions,btc_output_volume,btc_estimated_transaction_volume,target_t7,btc_trade_volume_log,btc_transaction_fees_log,btc_output_volume_log,btc_estimated_transaction_volume_log
175,2010-08-17,0.076900,3744250.0,923.0018,0.000959,0.0,0.003333,0.67,1.430952,393.0,352.0,72855.15,48276.0,0.066889,6.828714,0.512824,11.196242,10.784711
176,2010-08-18,0.074000,3750900.0,206.7786,0.001973,0.0,0.003384,1.56,1.305501,449.0,377.0,52829.65,35791.0,0.066500,5.336473,0.940007,10.874847,10.485480
177,2010-08-19,0.068800,3757900.0,51.8784,0.000715,0.0,0.003562,0.00,1.308696,395.0,368.0,32027.42,13756.0,0.066499,3.967995,0.000000,10.374379,9.529303
178,2010-08-20,0.066700,3766250.0,293.9825,0.000649,0.0,0.004249,0.00,1.399359,433.0,398.0,36647.52,21143.0,0.065000,5.686916,0.000000,10.509128,9.959111
179,2010-08-21,0.066899,3775450.0,731.0702,0.000528,0.0,0.004681,0.00,1.768594,396.0,348.0,33790.41,17456.0,0.065000,6.595876,0.000000,10.427962,9.767496
180,2010-08-22,0.066400,3785400.0,1118.9382,0.000491,0.0,0.005063,0.00,1.887657,358.0,350.0,45810.46,29971.0,0.064800,7.021029,0.000000,10.732290,10.308019
181,2010-08-23,0.066000,3796250.0,300.8243,0.000495,0.0,0.005521,0.00,1.840874,431.0,389.0,29274.46,11325.0,0.069000,5.709845,0.000000,10.284505,9.334856
182,2010-08-24,0.066889,3806500.0,469.8232,0.000434,0.0,0.005215,0.00,2.034458,393.0,337.0,28624.03,11639.0,0.064970,6.154483,0.000000,10.262037,9.362203
183,2010-08-25,0.066500,3817850.0,296.0216,0.000772,0.0,0.005775,0.80,1.905997,414.0,396.0,24234.76,7927.0,0.064900,5.693805,0.587787,10.095585,8.978156
184,2010-08-26,0.066499,3828250.0,270.9007,0.000499,0.0,0.005292,0.00,1.849170,391.0,374.0,30534.45,18406.0,0.062900,5.605437,0.000000,10.326644,9.820486


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2738 entries, 175 to 2912
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Date                                  2738 non-null   datetime64[ns]
 1   btc_market_price                      2738 non-null   float64       
 2   btc_total_bitcoins                    2738 non-null   float64       
 3   btc_trade_volume                      2738 non-null   float64       
 4   btc_avg_block_size                    2738 non-null   float64       
 5   btc_median_confirmation_time          2738 non-null   float64       
 6   btc_hash_rate                         2738 non-null   float64       
 7   btc_transaction_fees                  2738 non-null   float64       
 8   btc_cost_per_transaction              2738 non-null   float64       
 9   btc_n_unique_addresses                2738 non-null   float64       
 10  btc

In [17]:
df.describe(include="all").transpose()

,count,mean,min,25%,50%,75%,max,std
Date,2738,2014-05-16 12:00:00,2010-08-17 00:00:00,2012-07-01 06:00:00,2014-05-16 12:00:00,2016-03-30 18:00:00,2018-02-13 00:00:00,NaN
btc_market_price,2738.0,930.377008,0.0609,11.222022,252.71,617.921563,19498.683333,2419.879152
btc_total_bitcoins,2738.0,12059588.381118,3744250.0,9355512.5,12779187.5,15377793.75,16863312.5,3696172.889257
btc_trade_volume,2717.0,84993437.66292,41.0628,434593.971,11761058.7263,30866009.8532,5352015515.54,316929499.270474
btc_blocks_size,2738.0,38053.305789,1.0,1682.25,18247.5,63401.0,156580.256203,44618.160902
btc_avg_block_size,2738.0,0.376779,0.000434,0.076631,0.229098,0.715818,1.110327,0.354778
btc_n_transactions_per_block,2738.0,718.809712,1.0,175.25,411.5,1310.701923,2722.625,688.912493
btc_median_confirmation_time,2738.0,8.028042,0.0,6.683333,8.091667,10.579167,47.733333,4.724338
btc_hash_rate,2738.0,1423210.779134,0.003333,13.916822,71727.481822,1250355.732292,24293141.3193,3306748.836303
btc_difficulty,2738.0,185449574192.087036,511.773534,1866391.305,8853416309.129999,165496835118.0,2874674234420.0,420517264212.958008
